In [1]:
import pymongo
import json
import datetime
import pandas as pd
from pprint import pprint
from pymongo import MongoClient

In [2]:
client = MongoClient('mongo', 27017, username='admin', password='DataMan2019!')

In [3]:
db=client.UCLFinal

In [ ]:
#trova ed elimina i duplicati cercando per tweet id con questa funzione inserita da console mongo:

db.partita.aggregate([
 {
     "$group": {
         _id: { id : "$id"},
         dups: { $addToSet: "$_id" } ,
         count: { $sum : 1 } 
     }
 },
 {
     "$match": {
         count: { "$gt": 1 }
     }
 }
],{ allowDiskUse:true}).forEach(function(doc) {
   doc.dups.shift();
   db.partita.remove({
       _id: {$in: doc.dups}
   });
} )

In [21]:
#indexing, creiamo alcuni indici per facilitare la ricerca nei campi che utilizziamo per la ricerca. E' buona norma applicare al massimo 2 indici per collezione:

resp = db.partita.create_index([ ("timestamp", 1) ])
print ("index response:", resp) #non cambia il nome della variabile


#facilita la ricerca, creane per diversi campi, quelli più battuti. Il secondo indice, di default è '_id', fornito da mongo nell'atto dell'immagazzinamento

index response: timestamp_1


In [30]:
db.partita.estimated_document_count()   #numero di tweet nella collezione dopo l'eliminazione dei duplicati (solo 3 duplicati)

758373

Stat_UCL=pd.read_csv("/home/studente/my-data/statistiche")

In [31]:
#timestamp to ISODAte, tenendo anche timestamp:
db.partita.update_many(
   {"FixedDate": {"$exists": False}},
   [{"$set":{
      "FixedDate":{"$toString":"$timestamp"}
  }},
  {"$set":{
      "NewDate": {"$toDate": "$FixedDate"}
  }}
])

In [32]:
db.partita.find_one()

{'_id': ObjectId('5f42a3f3b957f1aa3f18ad40'),
 'id': '1297582966332362753',
 'user_location': None,
 'language': 'ar',
 'timestamp': '2020-08-23 19:14:21',
 'text': 'RT @realmeSaudi: ⚽ توقع واربح جهاز realme C3 🥳\nتوقع الفريق الرابح في المباراة اليوم  لفرصة الفوز بجهاز realme C3 \nشروط المسابقة بسيطة :\nـ م…',
 'hashtags': [],
 'urls': [],
 'user_mentions': [{'screen_name': 'realmeSaudi',
   'name': 'realme Saudi | ريلمي السعودية',
   'id': 1208760012635615237,
   'id_str': '1208760012635615237',
   'indices': [3, 15]}],
 'geo': None,
 'coordinates': None,
 'retweet_count': 0,
 'retweeted': False,
 'is_a_retweet': 'True',
 'user_followers': 48,
 'FixedDate': '2020-08-23 19:14:21',
 'NewDate': datetime.datetime(2020, 8, 23, 19, 14, 21)}

In [51]:
#EXPORT COLLEZIONI SUDDIVISE:

#PRIMO TEMPO:
agg=db.partita.aggregate(
   [{'$match':{"timestamp":{"$gte":"2020-08-23 21:01:00","$lt":"2020-08-23 21:48:00"}}},
     {
       '$project':
         {
             'id':'$id',
             'user_location':'$user_location',
             'language':'$language',
             'timestamp':'$timestamp',
             'text':'$text',
             'hashtags':'$hashtags',
             'urls':'$urls',
             'user_mentions':'$user_mentions',
             'geo':'$geo',
             'coordinates':'$coordinates',
             'retweet_count':'$retweet_count',
             'retweeted':'$retweeted',
             'is_a_retweet':'$is_a_retweet',
             'user_followers':'$user_followers',
             'minuto_partita': { '$minute': "$NewDate" }
         }
     },
       { '$out' :'primo_tempo'}   #out permette di ottenere una collezione nel db di riferimento
   ]
)

In [39]:
#SECONDO TEMPO:
agg=db.partita.aggregate(
   [{'$match':{"timestamp":{"$gte":"2020-08-23 22:04:00","$lte":"2020-08-23 22:54:00"}}},
     {
       '$project':
         {
             'id':'$id',
             'user_location':'$user_location',
             'language':'$language',
             'timestamp':'$timestamp',
             'text':'$text',
             'hashtags':'$hashtags',
             'urls':'$urls',
             'user_mentions':'$user_mentions',
             'geo':'$geo',
             'coordinates':'$coordinates',
             'retweet_count':'$retweet_count',
             'retweeted':'$retweeted',
             'is_a_retweet':'$is_a_retweet',
             'user_followers':'$user_followers',
             'minuto_partita': { '$add':[{'$minute': "$NewDate"}, 42]} #+45 minuti per entrare in secondo tempo e togliamo 3 per passare da 04 a 01 come pt
         }
     },
       { '$out' :'secondo_tempo'}
   ]
)

In [5]:
# INTERVALLO:
agg=db.partita.aggregate(
   [{'$match':{"timestamp":{"$lt":"2020-08-23 22:04:00","$gte":"2020-08-23 21:48:00"}}}, #riferimento dell'intervallo
     {
       '$project':
         {
             'id':'$id',
             'user_location':'$user_location',
             'language':'$language',
             'timestamp':'$timestamp',
             'text':'$text',
             'hashtags':'$hashtags',
             'urls':'$urls',
             'user_mentions':'$user_mentions',
             'geo':'$geo',
             'coordinates':'$coordinates',
             'retweet_count':'$retweet_count',
             'retweeted':'$retweeted',
             'is_a_retweet':'$is_a_retweet',
             'user_followers':'$user_followers',
             'minuto_partita': 'None' ,
             'minuto':'None',
             'testo':'None',
             'squadre':'None',
             'soggetti':'None',
             'eventi':'None'
         }
     },
       { '$out' :'intervallo'}
   ]
)

In [6]:
# PREPARTITA::
agg=db.partita.aggregate(
   [{'$match':{"timestamp":{"$lt":"2020-08-23 21:01:00"}}},
     {
       '$project':
         {
             'id':'$id',
             'user_location':'$user_location',
             'language':'$language',
             'timestamp':'$timestamp',
             'text':'$text',
             'hashtags':'$hashtags',
             'urls':'$urls',
             'user_mentions':'$user_mentions',
             'geo':'$geo',
             'coordinates':'$coordinates',
             'retweet_count':'$retweet_count',
             'retweeted':'$retweeted',
             'is_a_retweet':'$is_a_retweet',
             'user_followers':'$user_followers',
             'minuto_partita': 'None',
             'minuto':'None',
             'testo':'None',
             'squadre':'None',
             'soggetti':'None',
             'eventi':'None'
         }
     },
       { '$out' :'pre_partita'}
   ]
)

In [7]:
# POSTPARTITA::
agg=db.partita.aggregate(
   [{'$match':{"timestamp":{"$gt":"2020-08-23 22:54:00"}}},
     {
       '$project':
         {
             'id':'$id',
             'user_location':'$user_location',
             'language':'$language',
             'timestamp':'$timestamp',
             'text':'$text',
             'hashtags':'$hashtags',
             'urls':'$urls',
             'user_mentions':'$user_mentions',
             'geo':'$geo',
             'coordinates':'$coordinates',
             'retweet_count':'$retweet_count',
             'retweeted':'$retweeted',
             'is_a_retweet':'$is_a_retweet',
             'user_followers':'$user_followers',
             'minuto_partita': 'None', 
             'minuto':'None',
             'testo':'None',
             'squadre':'None',
             'soggetti':'None',
             'eventi':'None'
         }
     },
       { '$out' :'post_partita'}
   ]
)

In [ ]:
#ora si convertono in CSV le collezioni

In [4]:
#girare json in csv:
tweet_df = pd.DataFrame(list(db.primo_tempo.find()))
#tweet_df.tail()

In [5]:
tweet_df2 = pd.DataFrame(list(db.secondo_tempo.find()))
#tweet_df2.tail()

In [6]:
#importiamo la cronaca
racconto=pd.read_csv('/data/my-data/statistiche/FinaleCronaca.csv',encoding='ISO-8859-1')
df=pd.DataFrame(racconto)
df=df.drop("Unnamed: 0",axis='columns')
#df

In [7]:
#creiamo dataset primo e secondo tempo per evityare duplicati sui minuti di recupero e inizio secondo tempo
pt=df.loc[1:48]
st=df.loc[49:]

In [8]:
#lavoriamo sul primo tempo separatamente dal secondo tempo:
gp_pt=pt.groupby("minuto_effettivo").size().reset_index()
un=pd.merge(pt,gp_pt,left_on="minuto_effettivo",right_on="minuto_effettivo",how='left')
un_pt=un.rename(columns={un.columns[6]:'count'},inplace=False)

In [9]:
listone=un_pt.values.tolist() 
#ottengo una lista di liste dove ogni sottolista è una riga
li_minuto=[]  #creiamo liste vuote che verranno poi compilate e unite come colonne di un dataframe
li_minutoeff=[]
li_testo=[]
li_squadre=[]
li_soggetti=[]
li_eventi=[]
i=0
while i<len(listone):
    if listone[i][6]!=2:  #count diverso da 2 
        li_minuto.append(listone[i][0])
        li_minutoeff.append(listone[i][1])
        li_testo.append(listone[i][2])
        li_squadre.append(listone[i][3])
        li_soggetti.append(listone[i][4])
        li_eventi.append(listone[i][5])
    
    elif listone[i][0]==listone[i+1][0]:   #evita 4 count di fila
        i+=1 #salta la prima riga e lavora sulla seconda, che avrà sicuramente un 2 sulla colonna count 
        #creiamo nuove liste da compilare:
        new_testo=[]
        new_squadre=[]
        new_soggetti=[]
        new_eventi=[]
        #le compiliamo con i record delle due righe consecutive
        new_testo.append(listone[i-1][2]) #prima riga con 2 a count
        new_testo.append(listone[i][2]) #seconda riga con 2 a count
        new_squadre.append(listone[i-1][3])
        new_squadre.append(listone[i][3])
        new_soggetti.append(listone[i-1][4])
        new_soggetti.append(listone[i][4])
        new_eventi.append(listone[i-1][5])
        new_eventi.append(listone[i][5])
        #compiliamo le colonne finali
        li_minuto.append(listone[i][0])  #minuto ed effettivo rerstano gli stessi nelle due liste
        li_minutoeff.append(listone[i][1])
        li_testo.append(new_testo)
        li_squadre.append(new_squadre)
        li_soggetti.append(new_soggetti)
        li_eventi.append(new_eventi)
    
    i+=1
    


primo=pd.DataFrame({'minuto':li_minuto,'minuto_effettivo':li_minutoeff,'testo':li_testo,'squadre':li_squadre,'soggetti':li_soggetti,'eventi':li_eventi})
#primo
        


In [10]:
#merge primo tempo:
primoT=pd.merge(tweet_df,primo,left_on='minuto_partita',right_on='minuto_effettivo',how='left')
#primoT

In [11]:
#ora lavoriamo sul secondo tempo
gp_st=st.groupby("minuto_effettivo").size().reset_index()
un_st=pd.merge(st,gp_st,left_on="minuto_effettivo",right_on="minuto_effettivo",how='left')
un_st=un_st.rename(columns={un_st.columns[6]:'count'},inplace=False)


In [12]:
listone_st=un_st.values.tolist() 
#ottengo un alista di listedove ogni sottolista è una riga
li_minuto=[]  #creiamo liste vuote che verranno poi compilate e unite come colonne di un dataframe
li_minutoeff=[]
li_testo=[]
li_squadre=[]
li_soggetti=[]
li_eventi=[]
i=0
while i<len(listone_st):
    if listone_st[i][6]!=2:  #count diverso da 2 
        li_minuto.append(listone_st[i][0])
        li_minutoeff.append(listone_st[i][1])
        li_testo.append(listone_st[i][2])
        li_squadre.append(listone_st[i][3])
        li_soggetti.append(listone_st[i][4])
        li_eventi.append(listone_st[i][5])
    
    elif listone_st[i][0]==listone_st[i+1][0]:   #evita 4 count di fila
        i+=1 #salta la prima riga e lavora sulla seconda, che avrà sicuramente un 2 sulla colonna count 
        #creiamo nuove liste da compilare:
        new_testo=[]
        new_squadre=[]
        new_soggetti=[]
        new_eventi=[]
        #le compiliamo con i record delle due righe consecutive
        new_testo.append(listone_st[i-1][2]) #prima riga con 2 a count
        new_testo.append(listone_st[i][2]) #seconda riga con 2 a count
        new_squadre.append(listone_st[i-1][3])
        new_squadre.append(listone_st[i][3])
        new_soggetti.append(listone_st[i-1][4])
        new_soggetti.append(listone_st[i][4])
        new_eventi.append(listone_st[i-1][5])
        new_eventi.append(listone_st[i][5])
        #compiliamo le colonne finali
        li_minuto.append(listone_st[i][0])  #minuto ed effettivo rerstano gli stessi nelle due liste
        li_minutoeff.append(listone_st[i][1])
        li_testo.append(new_testo)
        li_squadre.append(new_squadre)
        li_soggetti.append(new_soggetti)
        li_eventi.append(new_eventi)
    
    i+=1
    


secondo=pd.DataFrame({'minuto':li_minuto,'minuto_effettivo':li_minutoeff,'testo':li_testo,'squadre':li_squadre,'soggetti':li_soggetti,'eventi':li_eventi})
#secondo


In [13]:
secondoT=pd.merge(tweet_df2,secondo,left_on='minuto_partita',right_on='minuto_effettivo',how='left')
#secondoT

In [14]:
#CONCATENIAMO I DATASET DI PRIMO E SECONDO TEMPO:
match=pd.concat([primoT,secondoT]).reset_index()
match=match.drop(["index","minuto_effettivo"],axis="columns")
match.head()

,_id,id,user_location,language,timestamp,text,hashtags,urls,user_mentions,geo,...,retweet_count,retweeted,is_a_retweet,user_followers,minuto_partita,minuto,testo,squadre,soggetti,eventi
0,5f42bcf1b957f1aa3f1b647e,1297609802558955521,Anywhere Anyway,es,2020-08-23 21:01:00,... Le voy a Neymar y Mbappé.... y arrancó el ...,"[{'text': 'ChampionsLeague', 'indices': [66, 8...",[],[],None,...,0,False,False,10217,1,1',PSG che Ã¨ partito determinato attuando un pr...,psg,0,pressing
1,5f42bcf1b957f1aa3f1b6480,1297609802647048194,None,fr,2020-08-23 21:01:00,RT @Footballogue: [#UCLFinal] PSG🇫🇷 0-0 🇩🇪BAYE...,"[{'text': 'UCLFinal', 'indices': [19, 28]}]",[],"[{'screen_name': 'Footballogue', 'name': 'Foot...",None,...,0,False,True,1619,1,1',PSG che Ã¨ partito determinato attuando un pr...,psg,0,pressing
2,5f42bcf1b957f1aa3f1b6481,1297609802672222209,İstanbul,ca,2020-08-23 21:01:00,RT @SozcuSkor: #UCL finalinde ilk 11'ler:\n\n📌...,"[{'text': 'UCL', 'indices': [15, 19]}, {'text'...",[],"[{'screen_name': 'SozcuSkor', 'name': 'SKOR', ...",None,...,0,False,True,2510622,1,1',PSG che Ã¨ partito determinato attuando un pr...,psg,0,pressing
3,5f42bcf1b957f1aa3f1b6483,1297609802756116488,"Buenos Aires, Argentina",es,2020-08-23 21:01:00,#UCLfinal en marcha @PSG_espanol @FCBayernES,"[{'text': 'UCLfinal', 'indices': [0, 9]}]",[],"[{'screen_name': 'PSG_espanol', 'name': 'Paris...",None,...,0,False,False,3531,1,1',PSG che Ã¨ partito determinato attuando un pr...,psg,0,pressing
4,5f42bcf1b957f1aa3f1b6484,1297609802793865217,Europe,en,2020-08-23 21:01:00,Lets goo @FCBayernEN #ChampionsLeague #UCLFi...,"[{'text': 'ChampionsLeague', 'indices': [23, 3...",[],"[{'screen_name': 'FCBayernEN', 'name': 'FC Bay...",None,...,0,False,False,1347,1,1',PSG che Ã¨ partito determinato attuando un pr...,psg,0,pressing


In [15]:
#hashtags: 
hastag=match["hashtags"].values.tolist()
hashtags=[]
for li in hastag:
    aus=[]
    for diz in li:
        aus.append(diz['text'])
    hashtags.append(aus)
hashtags #unire a dataset ed eliminare 
match["hashtags"]=hashtags   #sovrascritta
#match["hashtags"]

In [16]:
#usermentions:  #teniamo screen name e id 
usermentions=match["user_mentions"].values.tolist()
for li in usermentions:
    for diz in li:
        del diz['id_str']
        del diz['indices']
        del diz['name']
match["user_mentions"]=usermentions
#match["user_mentions"]

In [17]:
#urls
urls=match["urls"].values.tolist()
for li in urls:
    for diz in li:
        del diz['indices']
match["urls"]=urls
#match["urls"]

In [19]:
match.to_csv("match",sep='\t',index=False)


In [ ]:
#creiamo il file integrato con intervallo:


In [20]:
#CLEANING PER LE ALTRE COLLEZIONI:        


In [18]:
#intervallo:
tweet_int = pd.DataFrame(list(db.intervallo.find()))
#hashtags: 
hastag=tweet_int["hashtags"].values.tolist()
hashtags=[]
for li in hastag:
    aus=[]
    for diz in li:
        aus.append(diz['text'])
    hashtags.append(aus)
#hashtags #unire a dataset ed eliminare 
tweet_int["hashtags"]=hashtags   #sovrascritta


In [19]:
#usermentions:  #teniamo screen name e id 
usermentions=tweet_int["user_mentions"].values.tolist()
for li in usermentions:
    for diz in li:
        del diz['id_str']
        del diz['indices']
        del diz['name']
tweet_int["user_mentions"]=usermentions

In [20]:
#urls
urls=tweet_int["urls"].values.tolist()
for li in urls:
    for diz in li:
        del diz['indices']
tweet_int["urls"]=urls

In [21]:
vertical_stack = pd.concat([match, tweet_int], axis=0)

In [22]:
match_int = vertical_stack.sort_values(by='timestamp',ascending=True)
#match_int.tail()

In [ ]:
match_int.to_csv("match_int",sep="\t",index=False)

In [4]:
#pre partita:
tweet_pre = pd.DataFrame(list(db.pre_partita.find()))
#hashtags: 
hastag=tweet_pre["hashtags"].values.tolist()
hashtags=[]
for li in hastag:
    aus=[]
    for diz in li:
        aus.append(diz['text'])
    hashtags.append(aus)
#hashtags #unire a dataset ed eliminare 
tweet_pre["hashtags"]=hashtags   #sovrascritta










In [5]:
#usermentions:  #teniamo screen name e id 
usermentions=tweet_pre["user_mentions"].values.tolist()
for li in usermentions:
    for diz in li:
        del diz['id_str']
        del diz['indices']
        del diz['name']
tweet_pre["user_mentions"]=usermentions


In [6]:
#urls
urls=tweet_pre["urls"].values.tolist()
for li in urls:
    for diz in li:
        del diz['indices']
tweet_pre["urls"]=urls

In [4]:
#post partita:
tweet_post = pd.DataFrame(list(db.post_partita.find()))




In [5]:
#hashtags: 
hastag=tweet_post["hashtags"].values.tolist()
hashtags=[]
for li in hastag:
    aus=[]
    for diz in li:
        aus.append(diz['text'])
    hashtags.append(aus)
#hashtags #unire a dataset ed eliminare 
tweet_post["hashtags"]=hashtags   #sovrascritta


In [6]:
#usermentions:  #teniamo screen name e id 
usermentions=tweet_post["user_mentions"].values.tolist()
for li in usermentions:
    for diz in li:
        del diz['id_str']
        del diz['indices']
        del diz['name']
tweet_post["user_mentions"]=usermentions

In [7]:
#urls
urls=tweet_post["urls"].values.tolist()
for li in urls:
    for diz in li:
        del diz['indices']
tweet_post["urls"]=urls

In [15]:
tweet_pre.to_csv("pre_partita",sep='\t',index=False)

In [16]:
tweet_post.to_csv("post_partita",sep='\t',index=False)

In [ ]:
completo=pd.concat([pre,match_int,post])
completo.drop_duplicates(subset='_id',inplace=True)
low_memory=False

In [ ]:
completo.to_csv("completo",sep="\t",index=False)  #si ottiene il file definitivo sulla partita completa, con pre e post partita